# Variable creation and dataframe editing

This notebook creates the following dataframes (in pickle format) for later use: company_details_pivoted, esg_urls_pd, full_report_details_df, hand_labelled_data, and paragraph_level_all.

## Set-up

### Import modules

In [1]:
import pandas as pd
import numpy as np
import re
import os
import math

### Functions

In [2]:
# need to extract the year from the variables (extract what's within [])
def get_year(var_name):
    '''
    Extracts year from variable names (year is in brackets either as FY#### or as 
    MM/DD/#### where #### is the year)
    '''
    find_pattern = re.findall(r'\[.*?\]', var_name)
    if len(find_pattern)==0 or len(find_pattern)>1:
        return None
    relevant_re = find_pattern[0]
    if 'FY ' in relevant_re:
        year = int(re.sub('FY |\[|\]', '', relevant_re))
    else:
        year = int(re.sub('\[|\]', '', relevant_re)[-4:])
    return year

## Data on report availability

In [3]:
# Import data from Excel
esg_urls_pd = pd.read_excel('../Data/report_links.xlsx')

# Remove colons from Exchange Ticker column
esg_urls_pd.rename(columns={'Exchange:Ticker':'Exchange-Ticker'},inplace=True)
esg_urls_pd['Exchange-Ticker'] = esg_urls_pd['Exchange-Ticker'].str.replace(':','-')
esg_urls_pd['Exchange-Ticker'] = esg_urls_pd['Exchange-Ticker'].str.upper()

# Find out how many years with reports there are per company
esg_urls_pd['Years with reports'] = esg_urls_pd.apply(lambda x: sum(x[[2012,2013,2014,
                                                                   2015,2016,2017,
                                                                   2018,2019,2020,2021]].notna()), axis=1)

In [4]:
# Check
esg_urls_pd.head(2)

,Company Name,Exchange-Ticker,Primary Sector,2012,2013,2014,2015,2016,2017,2018,...,2020,2021,Downloaded?,Multiple reports per year?,"No reports, but some info on website",In integrated reports,Non-English,No reports or info,Other notes,Years with reports
0,AB Electrolux (publ) (OM:ELUX B),OM-ELUX B,Consumer Discretionary,NaN,https://www.electroluxgroup.com/wp-content/upl...,https://www.electroluxgroup.com/wp-content/upl...,https://www.electroluxgroup.com/wp-content/upl...,https://www.electroluxgroup.com/wp-content/upl...,https://www.electroluxgroup.com/en/electrolux-...,https://www.electroluxgroup.com/wp-content/upl...,...,https://www.electroluxgroup.com/wp-content/upl...,https://www.electroluxgroup.com/wp-content/upl...,NaN,NaN,NaN,NaN,NaN,NaN,2012 only online: https://www.electroluxgroup....,9
1,Accor SA (ENXTPA:AC),ENXTPA-AC,Consumer Discretionary,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,Just on website: https://group.accor.com/en/co...,0


In [5]:
# Save pickle
esg_urls_pd.to_pickle('../Data/Variable dataframes/esg_urls_pd.pkl')

## Paragraph-level dataframe (all)

In [6]:
# Import paragraphs from pickle
paras_no_sents_df = pd.read_pickle('../Data/paragraph_df.pkl')

# Get boolean values for supply chain
sc_mapper = {'Yes':True,'No':False}
paras_no_sents_df['SC_bool'] = paras_no_sents_df['Supply_Chain'].map(sc_mapper)

# Import predictions for all paragraphs
predicted_labels = pd.read_pickle('../Data/paragraph_predictions.pkl')

# Create named columns for action and E/S
predicted_labels['Supply chain actions'] = predicted_labels.apply(lambda x: 'Both' if x['Process_action_pred']==1 & x['Market_action_pred']==1 else ('Process only' if x['Process_action_pred']==1 else ('Market only' if x['Market_action_pred']==1 else 'Neither action')), 
                                                                              axis=1)
predicted_labels['Supply chain E/S'] = predicted_labels.apply(lambda x: 'Both' if x['Environment_pred']==1 & x['Social_pred']==1 else ('Environment only' if x['Environment_pred']==1 else ('Social only' if x['Social_pred']==1 else 'Neither')), 
                                                                              axis=1)

# Get label combinations
predicted_labels['Environment_process'] = predicted_labels.apply(lambda x: 1 if ((x['Process_action_pred']==1) & (x['Environment_pred']==1)) else 0, axis=1)
predicted_labels['Social_process'] = predicted_labels.apply(lambda x: 1 if ((x['Process_action_pred']==1) & (x['Social_pred']==1)) else 0, axis=1)
predicted_labels['Environment_market'] = predicted_labels.apply(lambda x: 1 if ((x['Market_action_pred']==1) & (x['Environment_pred']==1)) else 0, axis=1)
predicted_labels['Social_market'] = predicted_labels.apply(lambda x: 1 if ((x['Market_action_pred']==1) & (x['Social_pred']==1)) else 0, axis=1)

# Clean and merge
predicted_labels.rename(columns={'Paragraph':'Cleaned paragraph'}, inplace=True)
predicted_labels.drop(columns=['Supply_Chain'], inplace=True)
paragraph_level_all = paras_no_sents_df.merge(predicted_labels, on=['Paragraph_Order','Filename'], how='outer')

# Get topics from topic model per SC paragraph
paragraphs_w_topics = pd.read_pickle('../Data/paragraphs_w_topics.pkl')
paragraphs_w_topics = paragraphs_w_topics[['Paragraph_Order','Filename','Perc_Contribution','Topic_Keywords',
                                           'Dominant_Topic_Named','Groups']]
paragraphs_w_topics.rename(columns={'Perc_Contribution':'Dominant_topic_perc_Contribution',
                                    'Groups':'Dominant_topic_group'})
paragraphs_w_topics = pd.get_dummies(paragraphs_w_topics, columns=['Dominant_Topic_Named'], prefix='', prefix_sep='')
paragraph_level_all = paragraph_level_all.merge(paragraphs_w_topics, on=['Paragraph_Order','Filename'], how='outer')

In [7]:
paragraph_level_all.head(2)

,Paragraph_Order,Filename,Paragraph,Supply_Chain,SC_bool,Cleaned paragraph,Process_action_pred,Market_action_pred,Social_pred,Environment_pred,...,Management systems,Materials & packaging,Plans and progress,Policies,Product quality,Resource usage,Risk assessments,Society,Store operations,Transportation & logistics
0,0,XTRA-ADS-2012-1,Green Company Performance Analysis 2012 The Gr...,No,False,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,XTRA-ADS-2012-1,Green Company Performance Analysis 2012 With t...,No,False,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# Get alt preds
svm_preds = pd.read_pickle('../Data/svm_alt_preds.pkl').drop(columns=['Paragraph','Supply_Chain'])
alt_bert_preds = pd.read_pickle('../Data/bert_alt_preds.pkl').drop(columns=['Paragraph','Supply_Chain'])
alt_bert_preds.rename(columns={'Market_action_pred':'Market_action_alt',
                               'Environment_pred':'Environment_alt'}, inplace=True)
paragraph_level_all = paragraph_level_all.merge(svm_preds, on=['Paragraph_Order','Filename'], how='outer')
paragraph_level_all = paragraph_level_all.merge(alt_bert_preds, on=['Paragraph_Order','Filename'], how='outer')

# Get alt label combinations
paragraph_level_all['Environment_process_alt'] = paragraph_level_all.apply(lambda x: 1 if ((x['Process_action_alt']==1) & (x['Environment_alt']==1)) else 0, axis=1)
paragraph_level_all['Social_process_alt'] = paragraph_level_all.apply(lambda x: 1 if ((x['Process_action_alt']==1) & (x['Social_alt']==1)) else 0, axis=1)
paragraph_level_all['Environment_market_alt'] = paragraph_level_all.apply(lambda x: 1 if ((x['Market_action_alt']==1) & (x['Environment_alt']==1)) else 0, axis=1)
paragraph_level_all['Social_market_alt'] = paragraph_level_all.apply(lambda x: 1 if ((x['Market_action_alt']==1) & (x['Social_alt']==1)) else 0, axis=1)

In [9]:
paragraph_level_all.head()

,Paragraph_Order,Filename,Paragraph,Supply_Chain,SC_bool,Cleaned paragraph,Process_action_pred,Market_action_pred,Social_pred,Environment_pred,...,Store operations,Transportation & logistics,Process_action_alt,Social_alt,Market_action_alt,Environment_alt,Environment_process_alt,Social_process_alt,Environment_market_alt,Social_market_alt
0,0,XTRA-ADS-2012-1,Green Company Performance Analysis 2012 The Gr...,No,False,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0
1,1,XTRA-ADS-2012-1,Green Company Performance Analysis 2012 With t...,No,False,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0
2,2,XTRA-ADS-2012-1,Table 1 shows the results of 2012 for energy c...,No,False,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0
3,3,XTRA-ADS-2012-1,We have set different targets for the adidas G...,No,False,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0
4,4,XTRA-ADS-2012-1,Two different baseline years for the reporting...,No,False,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0


In [10]:
paragraph_level_all.to_pickle('../Data/Variable dataframes/paragraph_level_all.pkl')

## Report-level details

In [11]:
# Import data from pickles
report_details_df = pd.read_pickle('../Data/report_details_df.pkl')
report_details_df['Exchange-Ticker'] = report_details_df['Exchange-Ticker'].str.upper()

In [12]:
# Get label counts and percentages at the report level
grouped = paragraph_level_all[paragraph_level_all['SC_bool']==True].drop(columns='Paragraph_Order').groupby('Filename').sum()
grouped.rename(columns={'Process_action_pred':'Process',
                        'Market_action_pred':'Market',
                        'Social_pred':'Social',
                        'Environment_pred':'Environment'}, inplace=True)
full_report_details_df = report_details_df.merge(grouped, left_index=True, right_index=True)

cols_to_calc_percent = ['Process','Market','Social','Environment',
                        'Environment_process','Social_process','Environment_market','Social_market', 
                        'Management systems','Deforestation',
                       'Human rights','Employee health & safety',
                       'Resource usage',
                       'Certifications & training',
                       'Collaborations & partnerships',
                       'Plans and progress',
                       'Governance & stakeholders','Policies',
                       'Product quality',
                       'Diversity & inclusion','Junk','Agriculture',
                       'Risk assessments','Chemicals',
                       'Transportation & logistics',
                       'Society','Store operations',
                       'Materials & packaging', 
                        'Market_action_alt','Process_action_alt',
                        'Environment_alt','Social_alt', 
                        'Environment_process_alt','Social_process_alt',
                        'Environment_market_alt','Social_market_alt']

for col in cols_to_calc_percent:
    full_report_details_df[f"{col}_percent"] = full_report_details_df.apply(lambda x: x[col] / x['Num_SC_paras'] * 100, axis=1)

full_report_details_df.head(2)

,Company Name,Exchange-Ticker,Primary Sector,Year,URL,to_download,dup_num,Num_paras,Num_SC_paras,Num_pages,...,Store operations_percent,Materials & packaging_percent,Market_action_alt_percent,Process_action_alt_percent,Environment_alt_percent,Social_alt_percent,Environment_process_alt_percent,Social_process_alt_percent,Environment_market_alt_percent,Social_market_alt_percent
XTRA-ADS-2012-1,adidas AG (XTRA:ADS),XTRA-ADS,Consumer Discretionary,2012,https://www.adidas-group.com/media/filer_publi...,True,1,27,1,NaN,...,0.0,0.000000,0.000000,100.000000,100.000000,0.000000,100.000000,0.000000,0.000000,0.000000
TSE-2802-2012-1,"Ajinomoto Co., Inc. (TSE:2802)",TSE-2802,Consumer Staples,2012,https://www.ajinomoto.co.jp/company/en/ir/libr...,True,1,431,30,118.0,...,0.0,16.666667,6.666667,46.666667,53.333333,46.666667,46.666667,16.666667,6.666667,3.333333


In [13]:
# Some page counts weren't captured with PyMuPDF; add these from Excel file
missing_page_counts = pd.read_excel('../Data/missing_page_counts.xlsx')
missing_page_counts.rename(columns={'Unnamed: 0':'Filename'}, inplace=True)
missing_page_counts.set_index('Filename', inplace=True)
full_report_details_df['Num_pages'] = full_report_details_df.apply(lambda x: missing_page_counts.loc[x.name]['Pages'] if x.name in missing_page_counts.index.tolist() else x['Num_pages'], axis=1)

In [14]:
full_report_details_df.to_pickle('../Data/Variable dataframes/full_report_details_df.pkl')

## Hand-labelled data

In [15]:
# Upload data labelled by hand
hand_labelled_data = pd.read_pickle('../Data/final_label_set.pkl')
hand_labelled_data.set_index('Index',inplace=True)

# Create named column
hand_labelled_data['Supply chain actions'] = hand_labelled_data.apply(lambda x: 'Both' if x['Process_action']==1 & x['Market_action']==1 else ('Process only' if x['Process_action']==1 else ('Market only' if x['Market_action']==1 else 'Neither action')), 
                                                                              axis=1)
hand_labelled_data['Supply chain E/S'] = hand_labelled_data.apply(lambda x: 'Both' if x['Environment']==1 & x['Social']==1 else ('Environment only' if x['Environment']==1 else ('Social only' if x['Social']==1 else 'Neither')), 
                                                                              axis=1)
hand_labelled_data.head(2)

,Filename,Paragraph_Order,Paragraph,Process_action,Market_action,Social,Environment,Animal welfare,Supply chain actions,Supply chain E/S
Index,,,,,,,,,,
69489,LSE-ABF-2015-1,453,"In common with all of the businesses, we have ...",1,0,0,0,0,Process only,Neither
104826,LSE-IHG-2016-1,113,. Procurement Hotels work to source goods and ...,0,0,0,0,0,Neither action,Neither


In [16]:
hand_labelled_data.to_pickle('../Data/Variable dataframes/hand_labelled_data.pkl')

## Company-level details

In [17]:
company_details = pd.read_excel('../Data/Company Screening Report.xlsx',header=7)

# Remove colons from Exchange Ticker column
company_details.rename(inplace=True, columns={'Exchange:Ticker':'Exchange-Ticker'})
company_details['Exchange-Ticker'] = company_details['Exchange-Ticker'].str.replace(':','-')
company_details['Exchange-Ticker'] = company_details['Exchange-Ticker'].str.upper()

# Get main industries
sectors = ['Automobiles and Components (Primary)',
           'Consumer Durables and Apparel (Primary)', 
           'Consumer Services (Primary)',
           'Retailing (Primary)',
           'Food and Staples Retailing (Primary)',
           'Food, Beverage and Tobacco (Primary)',
           'Household and Personal Products (Primary)']
company_details['Filtered industry'] = company_details['Industry Classifications'].apply(lambda x: [x for x in x.split('; ') if x in sectors])
company_details['Main industry'] = company_details['Filtered industry'].apply(lambda x: x[0] if len(set(x))==1 else ('Multiple sub-sectors' if len(set(x))>1 else None))

# Melt column since there is a column per variable per year (will need one column per variable and then a year col)
reference_columns = ['Company Name','Exchange-Ticker','Watch Lists','Primary Industry','Primary Sector',
                     'Industry Classifications','Year Founded','Main industry','Filtered industry',
                     'Activist Investors - % Owned [Latest] (%)', 
                     'Avg Price Volatility (#) - Capital IQ [Latest]','Geographic Region',
                     'Country/Region of Incorporation']
company_details_long = pd.melt(company_details, id_vars=reference_columns, var_name='Variable',
                               value_name='Value')

# Create a variable for sensitive industry (True/False)
sensitive_primary = ['Tobacco','Brewers','Casinos and Gaming', 'Tobacco', 'Distillers and Vintners']
company_details_long['Sensitive'] = company_details_long['Primary Industry'].apply(lambda x: True if x in sensitive_primary else False)

# Get year from variables
company_details_long['Year'] = company_details_long['Variable'].apply(get_year)

# Drop years from variables
company_details_long['Variable'] = company_details_long['Variable'].apply(lambda x: re.sub(r' \[.*?\]', '', x))

# Pivot data so variables are once again in separate columns but year is its own column
index_cols = reference_columns + ['Sensitive','Year']
index_cols.remove('Filtered industry')
company_details_pivoted = company_details_long.pivot(index=index_cols,columns='Variable',values='Value')
company_details_pivoted.reset_index(inplace=True)
company_details_pivoted.head(2)

# Set correct dtypes
numeric_cols = ['Year Founded',
                'Activist Investors - % Owned [Latest] (%)',
                'Avg Price Volatility (#) - Capital IQ [Latest]',
                'Year',
                'Book Value/Share ($USD, Historical rate)',
                'Market Capitalization ($USDmm, Historical rate)',
                'Return on Assets %',
                'Total Liabilities/Total Assets %',
                'Total Revenue ($USDmm, Historical rate)']
company_details_pivoted[numeric_cols] = company_details_pivoted[numeric_cols].apply(pd.to_numeric, errors='coerce')

# Get age of company
company_details_pivoted['Age'] = company_details_pivoted['Year'] - company_details_pivoted['Year Founded']

In [18]:
company_details_pivoted.head(2)

Variable,Company Name,Exchange-Ticker,Watch Lists,Primary Industry,Primary Sector,Industry Classifications,Year Founded,Main industry,Activist Investors - % Owned [Latest] (%),Avg Price Volatility (#) - Capital IQ [Latest],Geographic Region,Country/Region of Incorporation,Sensitive,Year,"Book Value/Share ($USD, Historical rate)","Market Capitalization ($USDmm, Historical rate)",Return on Assets %,Total Liabilities/Total Assets %,"Total Revenue ($USDmm, Historical rate)",Age
0,AB Electrolux (publ) (OM:ELUX B),OM-ELUX B,Consumer SP1200,Household Appliances,Consumer Discretionary,"Air Conditioners, Cooling (Primary); Climate C...",1901.0,Consumer Durables and Apparel (Primary),0.116,3.0,Europe,Sweden,False,2012,8.42,5673.5,3.96,79.1,16892.8,111.0
1,AB Electrolux (publ) (OM:ELUX B),OM-ELUX B,Consumer SP1200,Household Appliances,Consumer Discretionary,"Air Conditioners, Cooling (Primary); Climate C...",1901.0,Consumer Durables and Apparel (Primary),0.116,3.0,Europe,Sweden,False,2013,7.76,7304.5,3.24,81.2,16978.3,112.0


In [19]:
company_details['Main industry'].value_counts()

Food, Beverage and Tobacco (Primary)         50
Retailing (Primary)                          42
Consumer Durables and Apparel (Primary)      39
Automobiles and Components (Primary)         31
Consumer Services (Primary)                  30
Food and Staples Retailing (Primary)         24
Household and Personal Products (Primary)    16
Name: Main industry, dtype: int64

### Import Eikon data

In [20]:
# Import governance and controversies data from Eikon

gov_data_columns = ['Board Gender Diversity, Percent', 
                    'Sustainability Compensation Incentives',
                    'Executive Members Gender Diversity, Percent', 
                    'Ethnic Minorities Board Percentage',
                    'Board Cultural Diversity, Percent', 
                    'Global Compact Signatory']

# Different types of social controversies
workforce_controversies = ['Wages Working Condition Controversies Count',
                           'Diversity and Opportunity Controversies','Employees Health & Safety Controversies',
                           'Strikes']
human_rights_controversies = ['Human Rights Controversies','Child Labor Controversies']
community_controversies = ['Anti-Competition Controversies Count','Critical Countries Controversies',
                           'Intellectual Property Controversies','Business Ethics Controversies',
                           'Public Health Controversies','Tax Fraud Controversies']
product_responsibility_controversies = ['Consumer Complaints Controversies Count',
                                        'Controversies Customer Health & Safety','Controversies Privacy',
                                        'Controversies Product Access','Responsible Marketing Controversies',
                                        'Controversies Responsible R&D']

# All columns to get from Excl sheets
controversies_data_columns = (['ESG Controversies Score', 'Environmental Controversies Count'] 
                              + workforce_controversies + human_rights_controversies 
                              + community_controversies + product_responsibility_controversies)



gov_data = pd.DataFrame(columns=gov_data_columns+['Year','sp_company_id'])
controversies_data = pd.DataFrame(columns=controversies_data_columns+['Year','sp_company_id'])

# Map Eikon IDs to S&P IDs
eikon_sp_mapper = pd.read_excel('../Data/SP Eikon mapping.xlsx')[['Eikon','S&P']].set_index('Eikon').to_dict()
eikon_sp_mapper = eikon_sp_mapper['S&P']

folder = '../Data/Eikon files'

for file in os.listdir(folder):
    if file.endswith('.xlsx'):
        eikon_id = file[:-5].split(' ')[-1].upper() # get Eikon ID from filename
        sp_company_id = eikon_sp_mapper[eikon_id]
        filepath = folder + '/' + file
        temp_gov_data = pd.read_excel(filepath, sheet_name="Governance", header=2)
        temp_gov_data.rename(columns={'Unnamed: 0':'Variable'}, inplace=True)
        temp_gov_data = temp_gov_data.set_index('Variable').T
        temp_gov_data.columns = temp_gov_data.columns.str.strip()
        temp_gov_data = temp_gov_data[gov_data_columns]
        temp_gov_data = temp_gov_data.rename_axis('Year').reset_index()
        temp_gov_data['sp_company_id'] = sp_company_id.replace(':','-')
        gov_data = pd.concat([gov_data, temp_gov_data], ignore_index=True)

        temp_controversies_data = pd.read_excel(filepath, sheet_name="Controversies", header=2)
        temp_controversies_data.rename(columns={'Unnamed: 0':'Variable'}, inplace=True)
        temp_controversies_data = temp_controversies_data.set_index('Variable').T
        temp_controversies_data.columns = temp_controversies_data.columns.str.strip()
        temp_controversies_data = temp_controversies_data[controversies_data_columns]
        temp_controversies_data = temp_controversies_data.rename_axis('Year').reset_index()
        temp_controversies_data['sp_company_id'] = sp_company_id.replace(':','-')
        controversies_data = pd.concat([controversies_data, temp_controversies_data], ignore_index=True)

In [21]:
# Clean up governance data
perc_cols = ['Board Gender Diversity, Percent',
             'Board Cultural Diversity, Percent',
             'Ethnic Minorities Board Percentage',
             'Executive Members Gender Diversity, Percent']
for col in perc_cols:
    gov_data[col] = gov_data[col].apply(lambda x: x.replace('%','') if isinstance(x, str) else None)
for col in perc_cols:
    gov_data[col] = gov_data[col].apply(lambda x: None if x == '--' else x)
bool_mapper = {'FALSE':False,'TRUE':True,'--':None}
cols = ['Sustainability Compensation Incentives','Global Compact Signatory']
for col in cols:
    gov_data[col] = gov_data[col].apply(lambda x: bool_mapper[x])
numeric_cols = ['Board Gender Diversity, Percent',
                'Board Cultural Diversity, Percent',
                'Executive Members Gender Diversity, Percent',
                'Ethnic Minorities Board Percentage',
                'Year']
gov_data[numeric_cols] = gov_data[numeric_cols].apply(pd.to_numeric, errors='coerce')

# Clean up controversies data
count_cols = ['Environmental Controversies Count', 'Wages Working Condition Controversies Count',
              'Diversity and Opportunity Controversies','Employees Health & Safety Controversies',
              'Human Rights Controversies','Child Labor Controversies','Intellectual Property Controversies',
              'Public Health Controversies','Business Ethics Controversies','Tax Fraud Controversies',
              'Anti-Competition Controversies Count','Critical Countries Controversies',
              'Consumer Complaints Controversies Count','Controversies Customer Health & Safety',
              'Controversies Privacy','Controversies Product Access',
              'Controversies Responsible R&D']
for col in count_cols:
    controversies_data[col] = controversies_data[col].apply(lambda x: '0' if x == '--' else x)
numeric_cols = count_cols + ['Year']
controversies_data[numeric_cols] = controversies_data[numeric_cols].apply(pd.to_numeric, errors='coerce')

bool_mapper = {'FALSE':False,'TRUE':True,'--':False}
bool_cols = ['Strikes','Responsible Marketing Controversies']
for col in bool_cols:
    controversies_data[col] = controversies_data[col].apply(lambda x: bool_mapper[x])

In [22]:
eikon_data = controversies_data.merge(gov_data, on=['sp_company_id','Year'])
eikon_data.rename(columns={'sp_company_id':'Exchange-Ticker'}, inplace=True)

In [23]:
# Social controversies by sub-topic
eikon_data['Workforce_controversies_bool'] = eikon_data.apply(lambda x: True if sum(x[workforce_controversies]) > 0 else False, axis=1)
eikon_data['Human_rights_controversies_bool'] = eikon_data.apply(lambda x: True if sum(x[human_rights_controversies]) > 0 else False, axis=1)
eikon_data['Community_controversies_bool'] = eikon_data.apply(lambda x: True if sum(x[community_controversies]) > 0 else False, axis=1)
eikon_data['Product_controversies_bool'] = eikon_data.apply(lambda x: True if sum(x[product_responsibility_controversies]) > 0 else False, axis=1)

# Environment and social controversies
eikon_data['Env_controversies_bool'] = eikon_data['Environmental Controversies Count'].apply(lambda x: True if x > 0 else False)
eikon_data['Soc_controversies_bool'] = eikon_data.apply(lambda x: any(x[['Workforce_controversies_bool',
                                                                         'Human_rights_controversies_bool',
                                                                         'Community_controversies_bool',
                                                                         'Product_controversies_bool']]), axis=1)

# Any controversies
eikon_data['Controversies_all_bool'] = eikon_data.apply(lambda x: any(x[['Env_controversies_bool',
                                                                         'Soc_controversies_bool']]), axis=1)

In [24]:
eikon_data.head(2)

,ESG Controversies Score,Environmental Controversies Count,Wages Working Condition Controversies Count,Diversity and Opportunity Controversies,Employees Health & Safety Controversies,Strikes,Human Rights Controversies,Child Labor Controversies,Anti-Competition Controversies Count,Critical Countries Controversies,...,Ethnic Minorities Board Percentage,"Board Cultural Diversity, Percent",Global Compact Signatory,Workforce_controversies_bool,Human_rights_controversies_bool,Community_controversies_bool,Product_controversies_bool,Env_controversies_bool,Soc_controversies_bool,Controversies_all_bool
0,A+,0,0,0,0,False,0,0,0,0,...,NaN,16.67,False,False,False,False,False,False,False,False
1,D+,0,0,0,0,False,0,0,0,0,...,NaN,14.29,False,False,False,False,False,False,False,False


In [25]:
sp_eikon_mapper = {v.replace(':','-'):k for k,v in eikon_sp_mapper.items()}
total_list = list(eikon_data['Exchange-Ticker'].unique())
has_2011 = list(eikon_data[eikon_data['Year']==2011]['Exchange-Ticker'].unique())
to_download_sp = [co for co in total_list if co not in has_2011]
to_download_eikon = [sp_eikon_mapper[co] for co in to_download_sp]

In [26]:
for_eikon = pd.DataFrame(columns=eikon_data.columns, index=to_download_eikon)
for_eikon.head()

,ESG Controversies Score,Environmental Controversies Count,Wages Working Condition Controversies Count,Diversity and Opportunity Controversies,Employees Health & Safety Controversies,Strikes,Human Rights Controversies,Child Labor Controversies,Anti-Competition Controversies Count,Critical Countries Controversies,...,Ethnic Minorities Board Percentage,"Board Cultural Diversity, Percent",Global Compact Signatory,Workforce_controversies_bool,Human_rights_controversies_bool,Community_controversies_bool,Product_controversies_bool,Env_controversies_bool,Soc_controversies_bool,Controversies_all_bool
2331.HK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CARR.PA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2267.T,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BMWG.DE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7270.T,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
to_download_all = pd.DataFrame([sp_eikon_mapper[co] for co in total_list])
to_download_all.to_excel('../Data/for-eikon-download-all.xlsx')

In [28]:
for_eikon.to_excel('../Data/for-eikon-download.xlsx')

In [29]:
eikon2011data = pd.read_excel('../Data/eikon-downloaded.xlsx', 
                              sheet_name='2011 vals only')
eikon2011data.rename(columns={'Unnamed: 0':'Eikon ID'},inplace=True)
eikon2011data['Exchange-Ticker'] = eikon2011data['Eikon ID'].apply(lambda x: eikon_sp_mapper[x])
eikon2011data.drop(columns=['Eikon ID'], inplace=True)
eikon2011data['Exchange-Ticker'] = eikon2011data['Exchange-Ticker'].apply(lambda x: x.replace(':','-'))
eikon2011data['Year'] = 2011

# Fix percent columns
perc_cols = ['Board Gender Diversity, Percent',
             'Board Cultural Diversity, Percent',
             'Ethnic Minorities Board Percentage',
             'Executive Members Gender Diversity, Percent']
for col in perc_cols:
    eikon2011data[col] = eikon2011data[col].apply(lambda x: x.replace('%','') if isinstance(x, str) else None)
for col in perc_cols:
    eikon2011data[col] = eikon2011data[col].apply(lambda x: None if x == '--' else x)

# Fix bool columns
bool_cols = ['Sustainability Compensation Incentives','Global Compact Signatory', 
             'Strikes','Responsible Marketing Controversies']
for col in bool_cols:
    eikon2011data[col] = eikon2011data[col].astype(bool)
numeric_cols2 = ['Board Gender Diversity, Percent',
                'Board Cultural Diversity, Percent',
                'Executive Members Gender Diversity, Percent',
                'Ethnic Minorities Board Percentage']
eikon2011data[numeric_cols] = eikon2011data[numeric_cols].apply(pd.to_numeric, errors='coerce')

# Fix numeric columns
count_cols = ['Environmental Controversies Count', 'Wages Working Condition Controversies Count',
              'Diversity and Opportunity Controversies','Employees Health & Safety Controversies',
              'Human Rights Controversies','Child Labor Controversies','Intellectual Property Controversies',
              'Public Health Controversies','Business Ethics Controversies','Tax Fraud Controversies',
              'Anti-Competition Controversies Count','Critical Countries Controversies',
              'Consumer Complaints Controversies Count','Controversies Customer Health & Safety',
              'Controversies Privacy','Controversies Product Access',
              'Controversies Responsible R&D']
for col in count_cols:
    eikon2011data[col] = eikon2011data[col].apply(lambda x: '0' if x == '--' else x)
numeric_cols2 = ['Board Gender Diversity, Percent',
                'Board Cultural Diversity, Percent',
                'Executive Members Gender Diversity, Percent',
                'Ethnic Minorities Board Percentage',
                'Year']
numeric_cols = count_cols + numeric_cols2
eikon2011data[numeric_cols] = eikon2011data[numeric_cols].apply(pd.to_numeric, errors='coerce')

# Social controversies by sub-topic
eikon2011data['Workforce_controversies_bool'] = eikon2011data.apply(lambda x: True if sum(x[workforce_controversies]) > 0 else False, axis=1)
eikon2011data['Human_rights_controversies_bool'] = eikon2011data.apply(lambda x: True if sum(x[human_rights_controversies]) > 0 else False, axis=1)
eikon2011data['Community_controversies_bool'] = eikon2011data.apply(lambda x: True if sum(x[community_controversies]) > 0 else False, axis=1)
eikon2011data['Product_controversies_bool'] = eikon2011data.apply(lambda x: True if sum(x[product_responsibility_controversies]) > 0 else False, axis=1)

# Environment and social controversies
eikon2011data['Env_controversies_bool'] = eikon2011data['Environmental Controversies Count'].apply(lambda x: True if x > 0 else False)
eikon2011data['Soc_controversies_bool'] = eikon2011data.apply(lambda x: any(x[['Workforce_controversies_bool',
                                                                         'Human_rights_controversies_bool',
                                                                         'Community_controversies_bool',
                                                                         'Product_controversies_bool']]), axis=1)

# Any controversies
eikon2011data['Controversies_all_bool'] = eikon2011data.apply(lambda x: any(x[['Env_controversies_bool',
                                                                         'Soc_controversies_bool']]), axis=1)

eikon2011data.head()

,ESG Controversies Score,Environmental Controversies Count,Wages Working Condition Controversies Count,Diversity and Opportunity Controversies,Employees Health & Safety Controversies,Strikes,Human Rights Controversies,Child Labor Controversies,Anti-Competition Controversies Count,Critical Countries Controversies,...,Global Compact Signatory,Exchange-Ticker,Year,Workforce_controversies_bool,Human_rights_controversies_bool,Community_controversies_bool,Product_controversies_bool,Env_controversies_bool,Soc_controversies_bool,Controversies_all_bool
0,A+,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,...,False,SEHK-2331,2011,False,False,False,False,False,False,False
1,C,NaN,2.0,NaN,NaN,True,NaN,NaN,NaN,NaN,...,True,ENXTPA-CA,2011,False,False,False,False,False,False,False
2,A+,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,...,False,TSE-2267,2011,False,False,False,False,False,False,False
3,A+,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,...,True,XTRA-BMW,2011,False,False,False,False,False,False,False
4,A+,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,...,False,TSE-7270,2011,False,False,False,False,False,False,False


In [30]:
eikon_data = pd.concat([eikon_data, eikon2011data])

In [31]:
eikon_esg_score_num = pd.read_excel('../Data/eikon-downloaded.xlsx', 
                              sheet_name='Num ESG score vals only')
eikon_esg_score_num.rename(columns={'Unnamed: 0':'Eikon ID'},inplace=True)
eikon_esg_score_num['Exchange-Ticker'] = eikon_esg_score_num['Eikon ID'].apply(lambda x: eikon_sp_mapper[x])
eikon_esg_score_num = pd.melt(eikon_esg_score_num, id_vars=['Exchange-Ticker', 'Eikon ID'], var_name='Year', value_name='ESG_controversies_numeric')
eikon_esg_score_num.drop(columns=['Eikon ID'], inplace=True)
eikon_esg_score_num['Exchange-Ticker'] = eikon_esg_score_num['Exchange-Ticker'].apply(lambda x: x.replace(':','-'))
eikon_esg_score_num.head()

,Exchange-Ticker,Year,ESG_controversies_numeric
0,SEHK-2331,2011,100.000000
1,ENXTPA-CA,2011,34.782609
2,TSE-2267,2011,100.000000
3,BOVESPA-BRFS3,2011,100.000000
4,SWX-NESN,2011,78.000000


In [32]:
eikon_data_small = eikon_data[['Year','Exchange-Ticker','ESG Controversies Score','Workforce_controversies_bool',
                               'Human_rights_controversies_bool','Community_controversies_bool',
                               'Product_controversies_bool','Env_controversies_bool','Soc_controversies_bool',
                               'Controversies_all_bool','Board Gender Diversity, Percent',
                               'Sustainability Compensation Incentives','Executive Members Gender Diversity, Percent',
                               'Ethnic Minorities Board Percentage','Board Cultural Diversity, Percent',
                               'Global Compact Signatory']]
eikon_data_small = eikon_data_small.merge(eikon_esg_score_num, on=['Exchange-Ticker','Year'])
eikon_data_small.head(2)

,Year,Exchange-Ticker,ESG Controversies Score,Workforce_controversies_bool,Human_rights_controversies_bool,Community_controversies_bool,Product_controversies_bool,Env_controversies_bool,Soc_controversies_bool,Controversies_all_bool,"Board Gender Diversity, Percent",Sustainability Compensation Incentives,"Executive Members Gender Diversity, Percent",Ethnic Minorities Board Percentage,"Board Cultural Diversity, Percent",Global Compact Signatory,ESG_controversies_numeric
0,2021,SEHK-2331,A+,False,False,False,False,False,False,False,16.67,False,6.67,NaN,16.67,False,100.000000
1,2020,SEHK-2331,D+,False,False,False,False,False,False,False,14.29,False,6.67,NaN,14.29,False,26.315789


### Import NAIC data

In [33]:
# Get NAICS for global companies (via ISIN)
naics_global = pd.read_excel('../Data/NAICS codes.xlsx')
isin_to_sp_id = pd.read_excel('../Data/Exchange ticker to ISIN.xlsx')
global_naics_w_sp_id = isin_to_sp_id[['SP_EXCHANGE_TICKER','SP_ISIN']].merge(naics_global[['fyear','isin','naics']],
                                                                    left_on='SP_ISIN',right_on='isin', how='inner')
global_naics_w_sp_id.rename(columns={'SP_EXCHANGE_TICKER':'Exchange:Ticker'}, inplace=True)
global_naics_w_sp_id.head(2)

,Exchange:Ticker,SP_ISIN,fyear,isin,naics
0,TSE:7203,JP3633400001,2011,JP3633400001,336111
1,TSE:7203,JP3633400001,2012,JP3633400001,336111


In [34]:
# Get NAICS for North American companies (via CIK)
naics_north_amer = pd.read_excel('../Data/CIK to NAICS.xlsx')
cik_to_sp_id = pd.read_excel('../Data/cik_to_sp_id.xlsx')
cik_to_sp_id['CIK'] = cik_to_sp_id['CIK'].apply(lambda x: x.split('; '))
cik_to_sp_id = cik_to_sp_id.explode('CIK')
cik_to_sp_id['CIK'] = cik_to_sp_id['CIK'].apply(pd.to_numeric, errors='coerce')
north_amer_naics_w_sp_id = cik_to_sp_id[['Exchange:Ticker','CIK']].merge(naics_north_amer[['fyear','cik','naics']],
                                                                    left_on='CIK',right_on='cik', how='inner')
north_amer_naics_w_sp_id.head(2)

,Exchange:Ticker,CIK,fyear,cik,naics
0,OM:ELUX B,813810.0,2012.0,813810,3352
1,OM:ELUX B,813810.0,2013.0,813810,3352


In [35]:
# Three codes were missing ('TSX-CTC.A', 'TSX-DOL', 'TSX-WN') -- get these
missing_naics = pd.read_excel('../Data/Missing NAICS codes.xlsx')

In [36]:
cols = ['Exchange:Ticker','naics','fyear']
all_naics = pd.concat([global_naics_w_sp_id[cols],
                       north_amer_naics_w_sp_id[cols], missing_naics[cols]])

# Remove colons from Exchange Ticker column
all_naics.rename(inplace=True, columns={'Exchange:Ticker':'Exchange-Ticker','fyear':'Year'})
all_naics['Exchange-Ticker'] = all_naics['Exchange-Ticker'].str.replace(':','-')
all_naics['naics_2dig'] = all_naics['naics'].apply(lambda x: str(x)[:2])
all_naics.head(2)

,Exchange-Ticker,naics,Year,naics_2dig
0,TSE-7203,336111,2011.0,33
1,TSE-7203,336111,2012.0,33


In [37]:
len(all_naics['Exchange-Ticker'].unique())

232

In [38]:
# Drop year since it doesn't change and there are missnig vals
all_naics = all_naics.drop(columns='Year')
all_naics = all_naics.drop_duplicates()

### Import innovation data

In [39]:
def get_innovation_index(naics2dig, naics_inno_code_list, inno_var):
    if naics2dig in naics_inno_code_list:
        return inno_var.index[inno_var['NAICS code']==naics2dig[:2]][0]
    else:
        return None
    
def get_innovation_index_to_4(naics, naics_inno_code_list, inno_var):
    naics = str(naics)
    if naics[:4] in naics_inno_code_list:
        return inno_var.index[inno_var['NAICS code']==naics[:4]][0]
    elif (naics[:3] + '*') in naics_inno_code_list:
        return inno_var.index[inno_var['NAICS code']==(naics[:3]+'*')][0]
    elif naics[:3] in naics_inno_code_list:
        return inno_var.index[inno_var['NAICS code']==(naics[:3])][0]
    elif (naics[:2] + '*') in naics_inno_code_list:
        return inno_var.index[inno_var['NAICS code']==(naics[:2]+'*')][0]  
    if naics[:2] in naics_inno_code_list:
        return inno_var.index[inno_var['NAICS code']==naics[:2]][0]
    else:
        return None

In [40]:
def custom_split(x):
    if '–' in str(x):
        x1 = int(x.split('–')[0])
        x2 = int(x.split('–')[1])+1
        xlist = list(range(x1,x2))
    elif '-' in str(x):
        x1 = int(x.split('-')[0])
        x2 = int(x.split('-')[1])+1
        xlist = list(range(x1,x2))
    elif 'other' not in str(x):
        xlist = [x]
    else:
        xlist = [x.replace('other ','')+'*']
    return xlist

def get_innovation_vars(filename, years):
    innovation_variables = pd.read_excel(filename)
    innovation_variables = innovation_variables[innovation_variables['Level']>0]

    innovation_variables['NAICS code'] = innovation_variables['NAICS code'].apply(custom_split)
    innovation_variables = innovation_variables.explode('NAICS code').reset_index(drop=True)
    innovation_variables['NAICS code'] = innovation_variables['NAICS code'].astype(str)
    
    
    # NAICS filtered
    naics_temp_df = all_naics.copy()
    
    # Get index
    naics_inno_code_list = list(innovation_variables['NAICS code'].unique())
    naics_temp_df['innovation_index'] = naics_temp_df['naics_2dig'].apply(lambda x: get_innovation_index(x, naics_inno_code_list, innovation_variables))
    naics_temp_df['innovation_index4'] = naics_temp_df['naics'].apply(lambda x: get_innovation_index_to_4(x, naics_inno_code_list, innovation_variables))
    
    # Get NAICS df
    naics_w_inno = naics_temp_df.merge(innovation_variables, 
                                   left_on='innovation_index', 
                                   right_index=True, how='left')
    naics_w_inno.drop(columns=['Level','innovation_index','Industry','Count of companies'], 
                      inplace=True)
    all_possible_cols = ['Product_or_process_perc_y','Product_perc_y','Process_perc_y']
    available_cols = [col for col in all_possible_cols if col in list(naics_w_inno.columns)]
    naics_w_inno = naics_w_inno.merge(innovation_variables[available_cols], 
                                      left_on='innovation_index4', 
                                      right_index=True, 
                                      suffixes = (None, '_detailed'), how='left')
    naics_w_inno.drop(columns=['innovation_index4'], inplace=True)
    naics_w_inno = naics_w_inno.drop_duplicates()
    
    # Fill for each year
    naics_w_inno_final = pd.DataFrame(columns=list(naics_w_inno.columns)+['Year'])
    for year in years:
        year_temp_df = naics_w_inno.copy()
        year_temp_df['Year'] = year
        naics_w_inno_final = pd.concat([naics_w_inno_final, year_temp_df])
    
    return naics_w_inno_final

In [41]:
inno_18_21 = get_innovation_vars('../Data/NSF_inno_2018-21.xlsx', [2018,2019,2020,2021])
inno_15_17 = get_innovation_vars('../Data/NSF_inno_2015-17.xlsx', [2015,2016,2017])
inno_12_14 = get_innovation_vars('../Data/NSF_inno_2012-14.xlsx', [2012,2013,2014])
inno_09_11 = get_innovation_vars('../Data/NSF_inno_2009-11.xlsx', [2011])

In [42]:
naics_w_inno = pd.concat([inno_18_21,inno_15_17,inno_12_14,inno_09_11])

In [43]:
naics_w_inno.head()

,Exchange-Ticker,naics,naics_2dig,NAICS code,Product_or_process_perc_y,Product_perc_y,Process_perc_y,Product_or_process_perc_y_detailed,Product_perc_y_detailed,Process_perc_y_detailed,Year
0,TSE-7203,336111,33,33,35.5,21.6,26.6,33.5,22.3,24.2,2018
10,TSE-7201,336111,33,33,35.5,21.6,26.6,33.5,22.3,24.2,2018
20,LSE-ULVR,3256,32,32,35.5,21.6,26.6,41.1,28.0,32.5,2018
30,LSE-BATS,312230,31,31,35.5,21.6,26.6,45.9,29.8,34.0,2018
40,KOSE-A005380,336111,33,33,35.5,21.6,26.6,33.5,22.3,24.2,2018


### Import RepRisk data

In [44]:
reprisk = pd.read_csv('../Data/reprisk.csv')
reprisk_extra = pd.read_csv('../Data/reprisk2.csv') # a few were missing so I downloaded them separately

reprisk = pd.concat([reprisk, reprisk_extra]).drop_duplicates(ignore_index=True)

# Data is daily -> need to convert to yearly
reprisk['Year'] = reprisk['date'].apply(lambda x: int(str(x)[:4]))
reprisk_grouped = reprisk.groupby(['reprisk_id','Year']).agg({'current_rri':['mean','min','max']}).reset_index()

# Flatten multi-index columns
reprisk_grouped.columns = ['_'.join(col) if col[1].strip() else col[0] for col in reprisk_grouped.columns]
reprisk_grouped.columns = reprisk_grouped.columns.to_flat_index()
reprisk_grouped.head(2)

# Connect Exchange Ticker to RepRisk ID
reprisk_id_to_isin = pd.read_excel('../Data/Reprisk ID to ISIN.xlsx')
reprisk_id_to_exchange_ticker = isin_to_sp_id[['SP_EXCHANGE_TICKER','SP_ISIN']].merge(reprisk_id_to_isin, 
                                                                                      right_on='primary_isin', 
                                                                                      left_on = 'SP_ISIN')
# Remove colons from Exchange Ticker column
reprisk_id_to_exchange_ticker.rename(inplace=True, columns={'SP_EXCHANGE_TICKER':'Exchange-Ticker'})
reprisk_id_to_exchange_ticker['Exchange-Ticker'] = reprisk_id_to_exchange_ticker['Exchange-Ticker'].str.replace(':','-')

reprisk_id_to_exchange_ticker.head(2)

,Exchange-Ticker,SP_ISIN,reprisk_id,primary_isin
0,NYSE-F,US3453708600,60,US3453708600
1,TSE-7203,JP3633400001,109,JP3633400001


In [45]:
len(reprisk_id_to_exchange_ticker['Exchange-Ticker'].unique())

231

In [46]:
# Identify missing company & verify data is not available for it
x1 = set(reprisk_id_to_exchange_ticker['Exchange-Ticker'].unique())
x2 = set(company_details_pivoted['Exchange-Ticker'].unique())
x2.difference(x1)

{'ENXTPA-ML'}

In [47]:
# Add Exchange-Ticker to overall reprisk data
reprisk_grouped = reprisk_grouped.merge(reprisk_id_to_exchange_ticker, on='reprisk_id')
reprisk_grouped.head(2)

,reprisk_id,Year,current_rri_mean,current_rri_min,current_rri_max,Exchange-Ticker,SP_ISIN,primary_isin
0,22,2012,28.715847,20,44,LSE-BATS,GB0002875804,GB0002875804
1,22,2013,26.241096,21,39,LSE-BATS,GB0002875804,GB0002875804


### Create company-level variables

In [48]:
cols = ['Num_paras','Num_SC_paras','Num_pages','Process','Market','Social','Environment',
        'Environment_process','Social_process','Environment_market','Social_market',
        'Management systems','Deforestation','Human rights','Employee health & safety',
        'Resource usage','Certifications & training','Collaborations & partnerships',
        'Plans and progress','Governance & stakeholders','Policies','Product quality',
        'Diversity & inclusion','Junk','Agriculture','Risk assessments','Chemicals',
        'Transportation & logistics',
        'Society','Store operations','Materials & packaging', 
        'Market_action_alt','Process_action_alt','Environment_alt','Social_alt',
        'Environment_process_alt','Social_process_alt','Environment_market_alt','Social_market_alt']

grouped_reports = full_report_details_df.groupby(['Exchange-Ticker','Year'])[cols].sum()
grouped_reports['Share_SC'] = grouped_reports['Num_SC_paras'] / grouped_reports['Num_paras'] * 100
for col in cols[3:]:
    grouped_reports[f"{col}_percent"] = grouped_reports.apply(lambda x: x[col] / x['Num_SC_paras'] * 100, axis=1)

grouped_reports = grouped_reports.reset_index()

In [49]:
grouped_reports.head(2)

,Exchange-Ticker,Year,Num_paras,Num_SC_paras,Num_pages,Process,Market,Social,Environment,Environment_process,...,Store operations_percent,Materials & packaging_percent,Market_action_alt_percent,Process_action_alt_percent,Environment_alt_percent,Social_alt_percent,Environment_process_alt_percent,Social_process_alt_percent,Environment_market_alt_percent,Social_market_alt_percent
0,ASX-ALL,2021,179,19,91.0,10.0,1.0,13.0,7.0,2.0,...,0.000000,0.000000,5.263158,21.052632,31.578947,68.421053,21.052632,5.263158,0.000000,5.263158
1,ASX-COL,2019,517,99,39.0,54.0,13.0,47.0,37.0,15.0,...,5.050505,7.070707,9.090909,29.292929,37.373737,45.454545,27.272727,8.080808,8.080808,1.010101


### Merging

In [50]:
company_details_pivoted = company_details_pivoted.merge(eikon_data_small, on=['Exchange-Ticker','Year'], how='left')
company_details_pivoted = company_details_pivoted.merge(reprisk_grouped, on=['Exchange-Ticker','Year'], how='left')
company_details_pivoted = company_details_pivoted.merge(naics_w_inno, on=['Exchange-Ticker','Year'], how='left')
company_details_pivoted = company_details_pivoted.merge(grouped_reports, on=['Exchange-Ticker','Year'], how='outer')

In [51]:
company_details_pivoted.head(2)

,Company Name,Exchange-Ticker,Watch Lists,Primary Industry,Primary Sector,Industry Classifications,Year Founded,Main industry,Activist Investors - % Owned [Latest] (%),Avg Price Volatility (#) - Capital IQ [Latest],...,Store operations_percent,Materials & packaging_percent,Market_action_alt_percent,Process_action_alt_percent,Environment_alt_percent,Social_alt_percent,Environment_process_alt_percent,Social_process_alt_percent,Environment_market_alt_percent,Social_market_alt_percent
0,AB Electrolux (publ) (OM:ELUX B),OM-ELUX B,Consumer SP1200,Household Appliances,Consumer Discretionary,"Air Conditioners, Cooling (Primary); Climate C...",1901.0,Consumer Durables and Apparel (Primary),0.116,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AB Electrolux (publ) (OM:ELUX B),OM-ELUX B,Consumer SP1200,Household Appliances,Consumer Discretionary,"Air Conditioners, Cooling (Primary); Climate C...",1901.0,Consumer Durables and Apparel (Primary),0.116,3.0,...,0.0,0.0,0.0,0.0,40.0,20.0,0.0,0.0,0.0,0.0


In [52]:
# Rename columns
column_mapper = {'Primary Industry':'primary_industry',
                 'Primary Sector':'primary_sector', 
                 'Industry Classifications':'industry_classifications', 
                 'Main industry':'main_industry', 
                 'Geographic Region':'geo_region',
                 'Country/Region of Incorporation':'country',
                 'Activist Investors - % Owned [Latest] (%)':'activist_investors',
                 'Avg Price Volatility (#) - Capital IQ [Latest]':'avg_price_volatility', 
                 'Book Value/Share ($USD, Historical rate)':'book_val_per_share',
                 'Market Capitalization ($USDmm, Historical rate)':'market_cap', 
                 'Return on Assets %':'ROA',
                 'Total Liabilities/Total Assets %':'liabilities_to_assets',
                 'Total Revenue ($USDmm, Historical rate)':'total_revenue', 
                 'ESG Controversies Score':'esg_controversies_score', 
                 'Board Gender Diversity, Percent':'board_gender_diversity',
                 'Sustainability Compensation Incentives':'sustainability_comp_incentives',
                 'Executive Members Gender Diversity, Percent':'exec_gender_diversity',
                 'Ethnic Minorities Board Percentage':'board_ethnic_diversity',
                 'Board Cultural Diversity, Percent':'board_cultural_diversity', 
                 'Global Compact Signatory':'global_compact'}
company_details_pivoted.rename(columns=column_mapper, inplace=True)

### Outliers and variable transformation

In [53]:
def remove_outliers(x):
    '''
    Replace outliers with NaN in a series
    '''
    q_low = x.quantile(0.01)
    q_hi  = x.quantile(0.99)
    
    trimmed_x = x.apply(lambda x: x if ((x < q_hi) & (x > q_low)) else None)
    
    return trimmed_x

def remove_stdev_outliers(x):
    mean = x.mean()
    std = x.std()
    trimmed_x = x.apply(lambda i: i if abs(i-mean)<(3*x.std()) else None)
    return trimmed_x

In [54]:
# Enable ignoring of data when paragraph count is below 10
company_details_pivoted['Num_paras_limit10'] = company_details_pivoted['Num_paras'].apply(lambda x: x if x >= 10 else None)
company_details_pivoted['Num_SC_paras_limit10'] = company_details_pivoted['Num_SC_paras'].apply(lambda x: x if x >= 10 else None)

# When total revenue = 0, change to NaN (missing values)
company_details_pivoted['total_revenue'] = company_details_pivoted['total_revenue'].apply(lambda x: None if x==0 else x)

# Get log of company size (market cap, revenues)
company_details_pivoted['log_market_cap'] = company_details_pivoted['market_cap'].apply(math.log)
company_details_pivoted['log_total_revenue'] = company_details_pivoted['total_revenue'].apply(math.log)

# Stdev to remove outliers (3 away)
company_details_pivoted['trimmed_book_val_per_share'] = remove_stdev_outliers(company_details_pivoted['book_val_per_share'])
company_details_pivoted['trimmed_ROA'] = remove_stdev_outliers(company_details_pivoted['ROA'])
company_details_pivoted['trimmed_liabilities_to_assets'] = remove_stdev_outliers(company_details_pivoted['liabilities_to_assets'])

# Create boolean dependent variables
company_details_pivoted['process_bool'] = company_details_pivoted['Process_percent'].apply(lambda x: True if x > 0 else (False if x == 0 else None))
company_details_pivoted['market_bool'] = company_details_pivoted['Market_percent'].apply(lambda x: True if x > 0 else (False if x == 0 else None))
company_details_pivoted['env_process_bool'] = company_details_pivoted['Environment_process_percent'].apply(lambda x: True if x > 0 else (False if x == 0 else None))
company_details_pivoted['soc_process_bool'] = company_details_pivoted['Social_process_percent'].apply(lambda x: True if x > 0 else (False if x == 0 else None))
company_details_pivoted['env_market_bool'] = company_details_pivoted['Environment_market_percent'].apply(lambda x: True if x > 0 else (False if x == 0 else None))
company_details_pivoted['soc_market_bool'] = company_details_pivoted['Social_market_percent'].apply(lambda x: True if x > 0 else (False if x == 0 else None))

# Categorical variables
company_details_pivoted["Year"] = company_details_pivoted["Year"].astype("category")
company_details_pivoted['main_industry'] = company_details_pivoted['main_industry'].astype("category")

### Get lagged variables

In [55]:
def create_lagged_columns(df, cols_to_lag, lag=1):
    '''
    Returns dataframe with lagged columns of interest (cols_to_lag)
    '''
    
    lagged_cols = df.sort_values(by=['Year']).groupby('Exchange-Ticker')[cols_to_lag].shift().add_suffix('_lag1')
    df = df.merge(lagged_cols, left_index=True, right_index=True)
    return df

cols_to_lag = ['Controversies_all_bool', 'Env_controversies_bool', 
               'Soc_controversies_bool', 'board_gender_diversity', 
               'sustainability_comp_incentives', 'ESG_controversies_numeric',
               'Product_or_process_perc_y_detailed']
company_details_pivoted = create_lagged_columns(company_details_pivoted, cols_to_lag)

### Get speaking space index variables

In [56]:
# Original predictions
company_details_pivoted['Market_to_process'] = (company_details_pivoted['Market_percent'] + 1) / (company_details_pivoted['Process_percent'] + 1)
company_details_pivoted['Market_to_process_log'] = company_details_pivoted['Market_to_process'].apply(np.log)
company_details_pivoted['Env_market_to_process'] = (company_details_pivoted['Environment_market_percent'] + 1) / (company_details_pivoted['Environment_process_percent'] + 1)
company_details_pivoted['Env_market_to_process_log'] = company_details_pivoted['Env_market_to_process'].apply(np.log)
company_details_pivoted['Soc_market_to_process'] = (company_details_pivoted['Social_market_percent'] + 1) / (company_details_pivoted['Social_process_percent'] + 1)
company_details_pivoted['Soc_market_to_process_log'] = company_details_pivoted['Soc_market_to_process'].apply(np.log)
company_details_pivoted['Env_to_social'] = (company_details_pivoted['Environment_percent'] + 1) / (company_details_pivoted['Social_percent'] + 1)
company_details_pivoted['Env_to_social_log'] = company_details_pivoted['Env_to_social'].apply(np.log)

In [57]:
# Alternate predictions
company_details_pivoted['Market_to_process_alt'] = (company_details_pivoted['Market_action_alt_percent'] + 1) / (company_details_pivoted['Process_action_alt_percent'] + 1)
company_details_pivoted['Market_to_process_log_alt'] = company_details_pivoted['Market_to_process_alt'].apply(np.log)
company_details_pivoted['Env_market_to_process_alt'] = (company_details_pivoted['Environment_market_alt_percent'] + 1) / (company_details_pivoted['Environment_process_alt_percent'] + 1)
company_details_pivoted['Env_market_to_process_log_alt'] = company_details_pivoted['Env_market_to_process_alt'].apply(np.log)
company_details_pivoted['Soc_market_to_process_alt'] = (company_details_pivoted['Social_market_alt_percent'] + 1) / (company_details_pivoted['Social_process_alt_percent'] + 1)
company_details_pivoted['Soc_market_to_process_log_alt'] = company_details_pivoted['Soc_market_to_process_alt'].apply(np.log)
company_details_pivoted['Env_to_social_alt'] = (company_details_pivoted['Environment_alt_percent'] + 1) / (company_details_pivoted['Social_alt_percent'] + 1)
company_details_pivoted['Env_to_social_log_alt'] = company_details_pivoted['Env_to_social_alt'].apply(np.log)

### Check column list

In [58]:
for col in company_details_pivoted.columns:
    print(col)

Company Name
Exchange-Ticker
Watch Lists
primary_industry
primary_sector
industry_classifications
Year Founded
main_industry
activist_investors
avg_price_volatility
geo_region
country
Sensitive
Year
book_val_per_share
market_cap
ROA
liabilities_to_assets
total_revenue
Age
esg_controversies_score
Workforce_controversies_bool
Human_rights_controversies_bool
Community_controversies_bool
Product_controversies_bool
Env_controversies_bool
Soc_controversies_bool
Controversies_all_bool
board_gender_diversity
sustainability_comp_incentives
exec_gender_diversity
board_ethnic_diversity
board_cultural_diversity
global_compact
ESG_controversies_numeric
reprisk_id
current_rri_mean
current_rri_min
current_rri_max
SP_ISIN
primary_isin
naics
naics_2dig
NAICS code
Product_or_process_perc_y
Product_perc_y
Process_perc_y
Product_or_process_perc_y_detailed
Product_perc_y_detailed
Process_perc_y_detailed
Num_paras
Num_SC_paras
Num_pages
Process
Market
Social
Environment
Environment_process
Social_process
En

In [59]:
company_details_pivoted.describe().T

,count,mean,std,min,25%,50%,75%,max
Year Founded,2280.0,1933.074561,51.310245,1742.000000,1898.750000,1940.000000,1972.000000,2.012000e+03
activist_investors,2270.0,1.044899,3.594160,-0.374000,0.036000,0.146000,0.569000,3.445000e+01
avg_price_volatility,2200.0,3.058227,0.928598,1.000000,2.750000,3.000000,3.500000,5.000000e+00
book_val_per_share,2289.0,101.758700,1156.386587,-116.500000,4.550000,12.100000,27.200000,2.409780e+04
market_cap,2211.0,38424.812619,76543.918613,74.100000,9668.850000,17952.000000,38018.750000,1.731328e+06
...,...,...,...,...,...,...,...,...
Env_market_to_process_log_alt,1105.0,-2.052281,1.106193,-4.615121,-2.726919,-1.806274,-1.296060,0.000000e+00
Soc_market_to_process_alt,1105.0,0.427139,0.699167,0.009901,0.094203,0.270073,0.583911,1.766667e+01
Soc_market_to_process_log_alt,1105.0,-1.441480,1.150700,-4.615121,-2.362304,-1.309063,-0.538006,2.871680e+00
Env_to_social_alt,1105.0,4.028158,15.553660,0.009901,0.754702,1.051666,1.492611,1.010000e+02


In [60]:
company_details_pivoted.to_pickle('../Data/Variable dataframes/company_details_pivoted.pkl')